In [1]:
# coding: utf-8
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import pandas as pd
import os #Import os package

In [2]:
### Read in file
#%%
os.chdir('/home/yingjie/Desktop')

test = pd.DataFrame.from_csv('test.csv',index_col=None)
train = pd.DataFrame.from_csv('train.csv',index_col=None)

In [3]:
test.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


In [4]:
train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


In [5]:
sum(train.target==0) # Check zeros in training dataset

573518

In [6]:
# PRE-PROCESSING - FULL MODEL
# Remove id from both training and testing dataset
test_noid = test.loc[: , test.columns !='id']
train_noid = train.loc[: , train.columns !='id']

Y = train_noid[["target"]] #training set target
x = train_noid.loc[:,train_noid.columns !="target"] #training set predictors
x2 = test_noid.loc[: , test_noid.columns !='target'] # testing set predictors

#%%

In [7]:
# PRE-PROCESSING - SUBSET MODEL
# Training set and validation set - set up
train_RF = pd.DataFrame.sample(train_noid,frac=.8) # 20/80 split
valid_RF = train_noid[~train_noid.index.isin(train_RF.index)]

# Training set preparation
train_RF_noT = train_RF.loc[:,train_RF.columns !="target"] # training set predictors
train_RF_T = train_RF[["target"]]

# Validation set preparation
valid_RF_noT = valid_RF.loc[:,valid_RF.columns !="target"] # validation set predictors
valid_RF_T = valid_RF[["target"]]

#%%

In [ ]:
''' Did not use this part of the code
# Define unnormalized gini index and normalized gini index
def unnormalized_gini_index(ground_truth, predicted_probabilities):
    if (len(ground_truth) !=  len(predicted_probabilities)):
        stop("Actual and Predicted need to be equal lengths!")
    x = len(ground_truth)   
    gini_table = pd.DataFrame(index = list(range(1,x+1,1)), predicted_probabilities=predicted_probabilities, ground_truth=ground_truth)
    gini_table = gini.table[order(-gini_table.predicted_probabilities, gini_table.index), ]
    num_ground_truth_positivies = sum(gini_table.ground_truth)
    model_percentage_positives_accumulated = gini_table.ground_truth / num_ground_truth_positivies
    random_guess_percentage_positives_accumulated = 1 / nrow(gini_table)
    gini_sum = cumsum(model_percentage_positives_accumulated - random_guess_percentage_positives_accumulated)
    gini_index = sum(gini_sum) / nrow(gini_table) 
    return(gini_index)

def normalized_gini_index(ground_truth, predicted_probabilities):
    model_gini_index = unnormalized_gini_index(ground_truth, predicted_probabilities)
    optimal_gini_index = unnormalized_gini_index(ground_truth, ground_truth)
    return(model_gini_index / optimal_gini_index)

#%%
'''

In [8]:
len(train_RF) #476170
len(valid_RF) #119042

119042

In [ ]:
# Random Forest - Takes a very long time to run
#%%
clf = RandomForestClassifier(max_depth=200, max_features=20, n_estimators=1000, random_state=0)
clf.fit(train_RF_noT,train_RF_T) # Fit training dataset
preds = (clf.predict(valid_RF_noT)) # Predict using validation set
preds
#%%

/home/yingjie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [ ]:
predsfinal = pd.DataFrame(preds)
predsfinal.columns = ["target"]
predsfinal.head()
sum(predsfinal.target==0) # Check how many predicted values are zeros
len(predsfinal)

In [ ]:
predsfinal.to_csv("finalpreds.csv") # Write out predicted file to be used in gini index function in R
valid_RF_T.to_csv("validtarget.csv") # Write out target values in validation set to be used in gini index function in R